In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Lets import the csv file in pandas dataframe first
data = pd.read_csv('/kaggle/input/ttc4900/7allV03.csv', encoding='utf-8')

In [ ]:
#check data
data.head()

In [ ]:
print(len(data))

In [ ]:
#learn unique category 
print(data.category.unique())

In [ ]:
# convert string labels to integers for bert classification
data['labels'] = pd.factorize(data.category)[0]

In [ ]:
data.head()

In [ ]:
#data split for train and test
from sklearn.model_selection import train_test_split

train, test = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
train.reset_index(drop=True, inplace=True)
test.reset_index(drop=True, inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

Fasttext Text Classification


[https://fasttext.cc/docs/en/supervised-tutorial.html](http://)

In [ ]:
#Data prepare for fasttext format for train
train["label_format"]=0
for i in range(len(train)):
    train.label_format[i]="__label__"+str(train.category[i])+" "+str(train.text[i])

In [ ]:
#Data prepare for fasttext format for test
test["label_format"]=0
for i in range(len(test)):
    test.label_format[i]="__label__"+str(test.category[i])+" "+str(test.text[i])

In [ ]:
train.label_format.to_csv('fasttext_train.txt',index=None,header=None)

In [ ]:
test.label_format.to_csv('fasttext_test.txt',index=None,header=None)

In [ ]:
import fasttext

In [ ]:
#train classifier model
model = fasttext.train_supervised('fasttext_train.txt',epoch=50,lr=0.05,label_prefix='__label__',dim=300)

In [ ]:
#test fasttext model
model.test('fasttext_test.txt')

In [ ]:
result = model.test('fasttext_train.txt')
validation = model.test('fasttext_test.txt')


# DISPLAY ACCURACY OF TRAINED MODEL
text_line =  "accuracy:" + str(result[1])  + ",validation:" + str(validation[1]) + '\n' 
print(text_line)

Bert Text Classification with Bert Turkish Model

I used this notebook in this work, thanks
[https://www.kaggle.com/ayhanc/bert-multilingual-for-turkish-text-classification](http://) 

Examine simpletransformers in detail from github
[https://github.com/ThilinaRajapakse/simpletransformers](http://)

In [ ]:
!pip install simpletransformers

In [ ]:
from simpletransformers.classification import ClassificationModel

In [ ]:
#for bert text column should be string and label column should be int
train["text"]=train["text"].apply(lambda r: str(r))
train['labels']=train['labels'].astype(int)

In [ ]:
#create model with turkish bert
model = ClassificationModel('bert', 'dbmdz/bert-base-turkish-uncased', num_labels=7, 
                            args={'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 3, "train_batch_size": 64 , "fp16":False, "output_dir": "bert_model"})

In [ ]:
#train model
model.train_model(train)

In [ ]:
result, model_outputs, wrong_predictions = model.eval_model(test)

In [ ]:
predictions = model_outputs.argmax(axis=1)
actuals = test.labels.values

In [ ]:
from sklearn import metrics
print(metrics.classification_report(actuals, predictions, digits=3))

I compared Bert and Fasttext classifiers and achieved higher success with Bert :)